In this week, we generated a forecast model for the EPL and compared its performance to that of the bookmakers’ odds. While the bookmakers’ predictions were slightly better, the difference was not great, whether measured by number of correct predictions of the result, or Brier score which measures the overall accuracy of the probabilities. 

For the assignment we will repeat this exercise for the Bundesliga in the 2019/20 season. We will use the data for the season up to March 11 (the last game before the COVID-19 lockdown began) to predict the results for the games played when the season resumed in May.

To answer the questions you need to complete the following steps:

### Step 1: Data preparation

In [ ]:
# This allows us to show the full screen width

# from IPython.display import display, HTML

# display(HTML(data="""
# <style>
#     div#notebook-container    { width: 95%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>
# """))

In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 1.	Load the data

In [2]:
BLmod = pd.read_excel("Assignment Data/Week 3/Bundesliga prediction model (Assignment).xlsx")
BLmod

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,Tmhome,Tmaway
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,1.14,8.00,15.00,116.05,215.20
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,1.20,7.00,13.00,790.40,116.05
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,1.95,3.50,4.00,162.28,101.10
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,1.75,3.75,4.75,35.58,81.65
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,2.25,3.25,3.40,81.65,148.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,27/06/2020,302,27,6,2020,Dortmund,Hoffenheim,0,4,A,1.70,4.50,4.10,642.10,229.50
302,27/06/2020,303,27,6,2020,Leverkusen,Mainz,1,0,H,1.25,6.50,10.00,420.60,148.60
303,27/06/2020,304,27,6,2020,Ein Frankfurt,Paderborn,3,2,H,1.30,5.75,9.00,207.55,24.10
304,27/06/2020,305,27,6,2020,Augsburg,RB Leipzig,1,2,A,7.00,5.75,1.36,116.05,521.90


In [3]:
# define the predicted result based on the betting odds 

BLmod['B365res']= np.where((BLmod['B365H']<BLmod['B365D']) & (BLmod['B365H']<BLmod['B365A']), 'H',\
                          np.where((BLmod['B365D']<BLmod['B365H']) & (BLmod['B365D']<BLmod['B365A']), 'D',\
                                  np.where((BLmod['B365A']<BLmod['B365H']) & (BLmod['B365A']<BLmod['B365D']), 'A',"")))

In [ ]:
# 2.	Define the result (H, D, A) and the probabilities associated with bookmaker odds

In [4]:
# define the probabilities associated with bookmaker odds

BLmod['B365probH'] = 1/BLmod['B365H']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probD'] = 1/BLmod['B365D']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])
BLmod['B365probA'] = 1/BLmod['B365A']/(1/BLmod['B365H']+1/BLmod['B365D']+1/BLmod['B365A'])

In [ ]:
# 3.	Define a winvalue = 2 if the home team wins, 1 if the game is a draw and zero otherwise

In [ ]:
# 4.	Define a variable equal to H if the home team wins and A if the visiting team wins

In [5]:
# we now create a winvalue (for the home team, 2 = win, 1 = draw, 0 = loss) which we will need for the order logit regression.

BLmod['winvalue'] = np.where(BLmod['FTR'] == 'H', 2, (np.where(BLmod['FTR'] == 'D', 1, 0)))

In [ ]:
# 5.	Generate a crosstab to compare the predictions of the bookmaker with the actual outcomes

In [6]:
# optional crosstab showing how often the bookmaker odds correctly predict the result

pd.crosstab(BLmod['FTR'], BLmod['B365res'], dropna=True)

B365res,A,H
FTR,,
A,63,52
D,18,50
H,28,95


In [ ]:
# Q1: Based on the crosstab, what percentage of games did the bookmaker predict correctly over the entire season
# A1: 52% 

In [ ]:
# 6.	Define a variable equal to the logarithm of the ratio of the home team TMvalue to the visiting team TMvalue

In [7]:
# create the log of salary ratios 
BLmod['lhTMratio'] = np.log(BLmod['Tmhome']/BLmod['Tmaway'])

### Step 2: Estimate a logit model of home time wins depending on the log TMvalue ratio, using the data for gamenos 1 to 224 as the “training data”

In [ ]:
# 1.	Define a subset for games 1 to 224

In [8]:
BLtrain = BLmod[:224]
BLtrain

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,...,B365D,B365A,Tmhome,Tmaway,B365res,B365probH,B365probD,B365probA,winvalue,lhTMratio
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,...,8.00,15.00,116.05,215.20,H,0.820681,0.116947,0.062372,1,-0.617547
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,...,7.00,13.00,790.40,116.05,H,0.791304,0.135652,0.073043,2,1.918518
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,...,3.50,4.00,162.28,101.10,H,0.489083,0.272489,0.238428,2,0.473213
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,...,3.75,4.75,35.58,81.65,H,0.544933,0.254302,0.200765,0,-0.830658
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,...,3.25,3.40,81.65,148.60,H,0.424796,0.294089,0.281115,2,-0.598816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2020-07-03 00:00:00,220,7,3,2020,Freiburg,Union Berlin,3,1,H,...,3.40,3.10,113.98,35.58,H,0.413495,0.279718,0.306787,2,1.164239
220,2020-07-03 00:00:00,221,7,3,2020,Hertha,Werder Bremen,2,2,D,...,3.50,3.40,215.20,162.28,H,0.450928,0.270557,0.278515,1,0.282245
221,2020-08-03 00:00:00,222,8,3,2020,Bayern Munich,Augsburg,2,0,H,...,10.00,21.00,790.40,116.05,H,0.859212,0.095373,0.045415,2,1.918518
222,2020-08-03 00:00:00,223,8,3,2020,Mainz,Fortuna Dusseldorf,1,1,D,...,3.75,3.80,148.60,81.65,H,0.498339,0.252492,0.249169,1,0.598816


In [ ]:
# 2.	Import the ordered logistic regression package

In [9]:
from bevel.linear_ordinal_regression import OrderedLogit

ol = OrderedLogit()

In [ ]:
# 3.	Run the ordered logistic regression of winvalue on the log TMvalue ratio

In [10]:
ol.fit(BLtrain['lhTMratio'], BLtrain['winvalue'])
ol.print_summary()

n=224
                  beta  se(beta)      p  lower 0.95  upper 0.95     
attribute names                                                     
column_1        0.5981    0.1129 0.0000      0.3769      0.8193  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Somers' D = 0.278


In [ ]:
# Q2: From the ordered logit model, what is the coefficient of the TM ratio variable?
# A2: 0.5981 

In [11]:
#%% To get the coefficients and the intercepts
print(f'beta = {ol.coef_[0]:.4f}')
print(f'interceptAD = {ol.coef_[1]:.4f}')
print(f'interceptDH = {ol.coef_[2]:.4f}')

beta = 0.5981
interceptAD = -0.6734
interceptDH = 0.3356


In [ ]:
# Q3: In the ordered logit model, what is the best we can say about the statistical significance of the TM ratio variable?
# A3: It is statistically significant at the 1% level (p-value) 

### Step 3: Define the predicted probabilities and the predicted results, using the entire data set

In [ ]:
# 1.	Define the predicted probabilities from the logit regression

In [12]:
# Predicted probabilities

BLmod['predA'] = 1/(1+np.exp(-(ol.coef_[1]-ol.coef_[0]*BLmod['lhTMratio'])))
BLmod['predD'] = 1/(1+np.exp(-(ol.coef_[2]-ol.coef_[0]*BLmod['lhTMratio']))) - BLmod['predA']
BLmod['predH'] = 1 - BLmod['predA'] - BLmod['predD']

pd.set_option('display.max_columns', 50)
BLmod

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,Tmhome,Tmaway,B365res,B365probH,B365probD,B365probA,winvalue,lhTMratio,predA,predD,predH
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,1.14,8.00,15.00,116.05,215.20,H,0.820681,0.116947,0.062372,1,-0.617547,0.424580,0.244701,0.330720
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,1.20,7.00,13.00,790.40,116.05,H,0.791304,0.135652,0.073043,2,1.918518,0.139329,0.168148,0.692524
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,1.95,3.50,4.00,162.28,101.10,H,0.489083,0.272489,0.238428,2,0.473213,0.277598,0.235530,0.486872
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,1.75,3.75,4.75,35.58,81.65,H,0.544933,0.254302,0.200765,0,-0.830658,0.455981,0.240881,0.303138
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,2.25,3.25,3.40,81.65,148.60,H,0.424796,0.294089,0.281115,2,-0.598816,0.421845,0.244951,0.333204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,27/06/2020,302,27,6,2020,Dortmund,Hoffenheim,0,4,A,1.70,4.50,4.10,642.10,229.50,H,0.557907,0.210765,0.231327,0,1.028841,0.216066,0.214435,0.569499
302,27/06/2020,303,27,6,2020,Leverkusen,Mainz,1,0,H,1.25,6.50,10.00,420.60,148.60,H,0.759124,0.145985,0.094891,2,1.040424,0.214895,0.213908,0.571197
303,27/06/2020,304,27,6,2020,Ein Frankfurt,Paderborn,3,2,H,1.30,5.75,9.00,207.55,24.10,H,0.729644,0.164963,0.105393,2,2.153160,0.123335,0.155090,0.721575
304,27/06/2020,305,27,6,2020,Augsburg,RB Leipzig,1,2,A,7.00,5.75,1.36,116.05,521.90,A,0.135787,0.165306,0.698906,0,-1.503455,0.556231,0.218429,0.225340


In [ ]:
# 2.	Based on the predicted probability, define the predicted result (H, D or A)

In [ ]:
# 3.	Based on the predicted outcome, create a dummy variable = 1 when the prediction is correct, and zero otherwise

In [13]:
# Result prediction

BLmod['Maxprob']= BLmod[['predA','predD','predH']].max(axis=1)
BLmod['logitpred']= np.where(BLmod['Maxprob']==BLmod['predA'], 'A',\
                            np.where(BLmod['Maxprob']==BLmod['predD'], 'D', 'H'))
BLmod['logittrue']= np.where(BLmod['logitpred']==BLmod['FTR'], 1, 0)
BLmod

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,Tmhome,Tmaway,B365res,B365probH,B365probD,B365probA,winvalue,lhTMratio,predA,predD,predH,Maxprob,logitpred,logittrue
0,16/08/2019,1,16,8,2019,Bayern Munich,Hertha,2,2,D,1.14,8.00,15.00,116.05,215.20,H,0.820681,0.116947,0.062372,1,-0.617547,0.424580,0.244701,0.330720,0.424580,A,0
1,17/08/2019,2,17,8,2019,Dortmund,Augsburg,5,1,H,1.20,7.00,13.00,790.40,116.05,H,0.791304,0.135652,0.073043,2,1.918518,0.139329,0.168148,0.692524,0.692524,H,1
2,17/08/2019,3,17,8,2019,Wolfsburg,FC Koln,2,1,H,1.95,3.50,4.00,162.28,101.10,H,0.489083,0.272489,0.238428,2,0.473213,0.277598,0.235530,0.486872,0.486872,H,1
3,17/08/2019,4,17,8,2019,Werder Bremen,Fortuna Dusseldorf,1,3,A,1.75,3.75,4.75,35.58,81.65,H,0.544933,0.254302,0.200765,0,-0.830658,0.455981,0.240881,0.303138,0.455981,A,1
4,17/08/2019,5,17,8,2019,Freiburg,Mainz,3,0,H,2.25,3.25,3.40,81.65,148.60,H,0.424796,0.294089,0.281115,2,-0.598816,0.421845,0.244951,0.333204,0.421845,A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,27/06/2020,302,27,6,2020,Dortmund,Hoffenheim,0,4,A,1.70,4.50,4.10,642.10,229.50,H,0.557907,0.210765,0.231327,0,1.028841,0.216066,0.214435,0.569499,0.569499,H,0
302,27/06/2020,303,27,6,2020,Leverkusen,Mainz,1,0,H,1.25,6.50,10.00,420.60,148.60,H,0.759124,0.145985,0.094891,2,1.040424,0.214895,0.213908,0.571197,0.571197,H,1
303,27/06/2020,304,27,6,2020,Ein Frankfurt,Paderborn,3,2,H,1.30,5.75,9.00,207.55,24.10,H,0.729644,0.164963,0.105393,2,2.153160,0.123335,0.155090,0.721575,0.721575,H,1
304,27/06/2020,305,27,6,2020,Augsburg,RB Leipzig,1,2,A,7.00,5.75,1.36,116.05,521.90,A,0.135787,0.165306,0.698906,0,-1.503455,0.556231,0.218429,0.225340,0.556231,A,1


In [ ]:
# 4.	Define three dummy variables for actual outcomes (H, D, A)

In [14]:
BLfore = BLmod[224:306]

# Outcome value for calculating Brier Score

BLfore['Houtcome']= np.where(BLfore['FTR'] == 'H', 1, 0)
BLfore['Doutcome']= np.where(BLfore['FTR'] == 'D', 1, 0)
BLfore['Aoutcome']= np.where(BLfore['FTR'] == 'A', 1, 0)
BLfore

,Date,gameno,day,month,year,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,Tmhome,Tmaway,B365res,B365probH,B365probD,B365probA,winvalue,lhTMratio,predA,predD,predH,Maxprob,logitpred,logittrue,Houtcome,Doutcome,Aoutcome
224,16/05/2020,225,16,5,2020,RB Leipzig,Freiburg,1,1,D,1.30,5.50,9.00,521.90,113.98,H,0.724214,0.171178,0.104609,1,1.521453,0.170317,0.189894,0.639790,0.639790,H,0,0,1,0
225,16/05/2020,226,16,5,2020,Hoffenheim,Hertha,0,3,A,2.10,3.60,3.20,229.50,215.20,H,0.446512,0.260465,0.293023,0,0.064335,0.329190,0.244539,0.426271,0.426271,H,0,0,0,1
226,16/05/2020,227,16,5,2020,Ein Frankfurt,M'gladbach,1,3,A,2.90,3.60,2.25,207.55,266.50,A,0.323160,0.260323,0.416517,0,-0.250002,0.371955,0.246995,0.381050,0.381050,H,0,0,0,1
227,16/05/2020,228,16,5,2020,Fortuna Dusseldorf,Paderborn,0,0,D,2.10,3.60,3.25,81.65,24.10,H,0.448534,0.261645,0.289822,1,1.220230,0.197306,0.205380,0.597313,0.597313,H,0,0,1,0
228,16/05/2020,229,16,5,2020,Dortmund,Schalke 04,4,0,H,1.50,4.33,6.00,642.10,213.48,H,0.626401,0.216998,0.156600,2,1.101201,0.208825,0.211098,0.580077,0.580077,H,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,27/06/2020,302,27,6,2020,Dortmund,Hoffenheim,0,4,A,1.70,4.50,4.10,642.10,229.50,H,0.557907,0.210765,0.231327,0,1.028841,0.216066,0.214435,0.569499,0.569499,H,0,0,0,1
302,27/06/2020,303,27,6,2020,Leverkusen,Mainz,1,0,H,1.25,6.50,10.00,420.60,148.60,H,0.759124,0.145985,0.094891,2,1.040424,0.214895,0.213908,0.571197,0.571197,H,1,1,0,0
303,27/06/2020,304,27,6,2020,Ein Frankfurt,Paderborn,3,2,H,1.30,5.75,9.00,207.55,24.10,H,0.729644,0.164963,0.105393,2,2.153160,0.123335,0.155090,0.721575,0.721575,H,1,1,0,0
304,27/06/2020,305,27,6,2020,Augsburg,RB Leipzig,1,2,A,7.00,5.75,1.36,116.05,521.90,A,0.135787,0.165306,0.698906,0,-1.503455,0.556231,0.218429,0.225340,0.556231,A,1,0,0,1


In [ ]:
# Q4: In the logistic regression model, if the ratio of the TM values equaled one, then
# A4: The value of the constants alone would determine the probability of a win, draw or loss for the home team

### Step 4: For games played from May 2020, compare the bookmaker probabilities and model probabilities in terms of the mean number of successfully predicted outcomes and the Brier scores

In [ ]:
# 1.	Define the subset of games played from May onwards

In [ ]:
# BLfore = BLmod[224:306] # already did, see above

In [ ]:
# 2.	Define a dummy variable equal to 1 when the bookmaker result prediction is correct, and zero otherwise

In [15]:
BLfore['B365true']= np.where(BLfore['FTR'] == BLfore['B365res'], 1, 0)

In [ ]:
# 3.	Calculate the means for each of these variables

In [16]:
BLfore['B365true'].mean()

0.5487804878048781

In [ ]:
# Q5: Based on the bookmaker odds, what fraction of results were correctly predicted from game 224 onwards?
# A5: 55% 

In [17]:
BLfore['logittrue'].mean()

0.5

In [ ]:
# Q6: Based on the ordered logit model, what fraction of results were correctly predicted
# A6: 50% 

In [ ]:
# 4.	Define the Brier score for the bookmaker probabilities and the Brier score for the logit model probabilities

In [18]:
BLfore['BrierB365'] = (BLfore['B365probH'] - BLfore['Houtcome'])**2 +(BLfore['B365probD'] - BLfore['Doutcome'])**2 +(BLfore['B365probA'] - BLfore['Aoutcome'])**2
BLfore['Brierlogit'] = (BLfore['predH'] - BLfore['Houtcome'])**2 +(BLfore['predD'] - BLfore['Doutcome'])**2 + (BLfore['predA'] - BLfore['Aoutcome'])**2

In [ ]:
# 5.	Calculate the mean of each Brier score

In [19]:
BLfore['BrierB365'].mean()

0.5620787329712248

In [ ]:
# Q7: What was the Brier score derived from the bookmaker odds? 
# A7: 0.562 

In [20]:
BLfore['Brierlogit'].mean()

0.5939861306698453

In [ ]:
# Q8: What was the Brier score derived from the logistic model?
# A8: 0.594

In [ ]:
# Q9: A lower Brier score implies
# A9: The probabilities were closer to the actual outcomes 

In [ ]:
# Q10: Suppose that the ordered logit model were updated after every game in the season, which of the following is most likely to be true:
# A10: The ordered logit model would produce more reliable forecasts 